In [13]:
import pandas as pd
import numpy as np
import pdfplumber as scrapper
import PyPDF2
from io import StringIO
import os
import glob 
import warnings
warnings.filterwarnings('ignore')

In [3]:
os.getcwd()

'/Users/jennylee/CFFS-PyCharm/notebooks'

## 1. Data Cleaning

In [10]:
def construct_empty_conversion_df():
    columns = ["ConversionId", "Multiplier", "ConvertFromQty", "ConvertFromUom", 
               "ConvertToQty", "ConvertToUom"]
    df = pd.DataFrame(columns=columns)
    return df

In [63]:
outcome = pd.read_csv("data/AMS/preprocessed/outcome_test8.csv")
outcome

,ConversionId,ConvertToQty,ConvertToUom,Category,CO2 Emission (g CO2),Nitrogen Lost (g),Water Withdrawals (mL),PrepId
0,I-0,1000.0,g,38,0.3062,0.0079,9.9,P-0
1,I-2,1.0,g,55,0.0000,0.0000,1.0,P-1
2,I-77,5000.0,g,55,0.0000,0.0000,1.0,P-14
3,I-95,100.0,g,55,0.0000,0.0000,1.0,P-17
4,I-6,30.0,g,37,0.6932,0.0079,77.0,P-1
...,...,...,...,...,...,...,...,...
132,I-22,35.0,g,59,0.0000,0.0000,0.0,P-4
133,I-50,1.0,g,59,0.0000,0.0000,0.0,P-9
134,I-74,1.0,g,59,0.0000,0.0000,0.0,P-13
135,I-43,24.0,g,3,9.8315,0.1328,1810.3,P-8


In [66]:
outcome["ConvertToQty"].tolist()

['1000.0',
 '1.0',
 '5000.0',
 '100.0',
 '30.0',
 '30.0',
 '300.0',
 '80.51265.0',
 '45.0',
 '20.0',
 '2724.0',
 '2.84',
 '5.0',
 '17.0',
 '300.0',
 '750.0',
 '600.0',
 '350.0',
 '45.0',
 '10.0',
 '15.0',
 '15.0',
 '200.0',
 '200.0',
 '200.0',
 '15.0',
 '200.0',
 '500.0',
 '30.0',
 '16329.312',
 '1000.0',
 '15.0',
 '250.0',
 '150.0',
 '30.0',
 '100.0',
 '10.0',
 '30.0',
 '73.0',
 '5.0',
 '10.0',
 '750.0',
 '250.0',
 '120.0',
 '2.1',
 '5.0',
 '2267.96',
 '175.0',
 '450.0',
 '100.0',
 '30.0',
 '200.0',
 '320.0',
 '100.0',
 '30.0',
 '5.0',
 '12000.0',
 '30.0',
 '5.0',
 '3000.0',
 '2000.0',
 '250.0',
 '50.0',
 '600.0',
 '18.0',
 '1.0',
 '30.0',
 '150.0',
 '50.0',
 '30.0',
 '10.0',
 '15.0',
 '4.0',
 '16.0',
 '3.0',
 '100.0',
 '10.0',
 '2000.0',
 '300.0',
 '15.0',
 '1500.0',
 '600.0',
 '4000.0',
 '60.0',
 '2840.0',
 '100.0',
 '100.0',
 '30.0',
 '3000.0',
 '12.0',
 '60.0',
 '9071.84',
 '2267.96',
 '2267.96',
 '2267.96',
 '50.0',
 '4535.92',
 '30.0',
 '5.0',
 '100.0',
 '50.0',
 '15.0',
 '165.0

In [59]:
outcome.loc[outcome["PrepId"] == "P-5"]

,ConversionId,ConvertToQty,ConvertToUom,Category,CO2 Emission (g CO2),Nitrogen Lost (g),Water Withdrawals (mL),PrepId,Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL)
6,I-25,12.0,g,40,0.5029,0.0079,81.3,P-5,6.03480,0.09480,975.60
18,I-23,168.0,g,40,0.5029,0.0079,81.3,P-5,84.48720,1.32720,13658.40
75,I-24,120.0,g,39,0.3015,0.0079,1.9,P-5,36.18000,0.94800,228.00
76,I-26,2.1,g,39,0.3015,0.0079,1.9,P-5,0.63315,0.01659,3.99


In [38]:
outcome3 = pd.read_csv("data/AMS/preprocessed/Labelled_per_Items.csv")
outcome3

,ConversionId,ConvertToQty,ConvertToUom,Category,CO2 Emission (g CO2),Nitrogen Lost (g),Water Withdrawals (mL),PrepId,Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),Weight (g),GHG Emission (g) / 100g,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g,GHG Emission (g) / g,N lost (g) / g,Stress-Weighted Water Use (L) / g
0,I-0,1000.00,g,38,0.3062,0.0079,9.9,P-0,306.200000,7.900000,9900.000,1000.0000,30.620000,0.790000,0.990000,0.306200,0.007900,0.009900
1,I-89,15.00,g,38,0.3062,0.0079,9.9,P-16,4.593000,0.118500,148.500,1435.0000,0.320070,0.008258,0.010348,0.003201,0.000083,0.000103
2,I-126,5.00,g,38,0.3062,0.0079,9.9,P-23,1.531000,0.039500,49.500,18523.9600,0.008265,0.000213,0.000267,0.000083,0.000002,0.000003
3,I-2,1.00,g,40,0.5029,0.0079,81.3,P-1,0.502900,0.007900,81.300,93.5000,0.537861,0.008449,0.086952,0.005379,0.000084,0.000870
4,I-6,30.00,g,40,0.5029,0.0079,81.3,P-1,15.087000,0.237000,2439.000,93.5000,16.135829,0.253476,2.608556,0.161358,0.002535,0.026086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,I-152,22679.60,g,4,4.3996,0.1168,370.3,P-28,99781.168160,2648.977280,8398255.880,24779.6000,402.674652,10.690154,33.891814,4.026747,0.106902,0.338918
133,I-153,2100.00,g,4,4.3996,0.1168,370.3,P-28,9239.160000,245.280000,777630.000,24779.6000,37.285348,0.989846,3.138186,0.372853,0.009898,0.031382
134,I-35,56.00,g,3,9.8315,0.1328,1810.3,P-7,550.564000,7.436800,101376.800,3412.0000,16.136108,0.217960,2.971184,0.161361,0.002180,0.029712
135,I-124,2267.96,g,35,0.3942,0.0027,37.4,P-23,894.029832,6.123492,84821.704,18523.9600,4.826343,0.033057,0.457903,0.048263,0.000331,0.004579


In [34]:
outcome2 = pd.read_csv("data/AMS/preprocessed/Labelled_per_Child_Items.csv")
outcome2

,ProdId,Weight (g),Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),GHG Emission (g) / 100g,N lost (g) / 100g,Stress-Weighted Water Use (L) / 100g,GHG Emission (g) / g,N lost (g) / g,Stress-Weighted Water Use (L) / g
0,P-9,4835.92,1206.557084,17.916884,184385.148,24.949898,0.370496,3.812825,0.249499,0.003705,0.038128
1,P-5,600.00,0.000000,4.050000,12135.000,0.000000,0.675000,2.022500,0.000000,0.006750,0.020225
2,P-12,30.00,13.200000,0.000000,0.000,44.000000,0.000000,0.000000,0.440000,0.000000,0.000000
3,P-25,12000.00,83788.800000,336.960000,336960.000,698.240000,2.808000,2.808000,6.982400,0.028080,0.028080
4,P-21,6930.00,17677.842500,63.924500,2363388.500,255.091522,0.922431,34.103730,2.550915,0.009224,0.341037
5,P-18,750.00,226.125000,5.925000,1425.000,30.150000,0.790000,0.190000,0.301500,0.007900,0.001900
6,P-27,175.00,52.762500,1.382500,332.500,30.150000,0.790000,0.190000,0.301500,0.007900,0.001900
7,P-17,3000.00,9720.300000,0.600000,4200.000,324.010000,0.020000,0.140000,3.240100,0.000200,0.001400
8,P-15,15.00,24.621000,0.000000,151.500,164.140000,0.000000,1.010000,1.641400,0.000000,0.010100
9,P-2,1082.00,1647.345000,16.013600,453574.400,152.250000,1.480000,41.920000,1.522500,0.014800,0.419200


In [50]:
conv = pd.read_csv("data/AMS/conversions/Converted_Units12.csv")
conv

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,I-0,1.0,1000.00,g,1000.0,g
1,I-2,1.0,1.00,g,1.0,g
2,I-6,1.0,30.00,g,30.0,g
3,I-7,1.0,45.00,g,45.0,g
4,I-8,1.0,15.00,g,15.0,g
...,...,...,...,...,...,...
137,I-3,0.0,2.00,PORT,4735.92,g
138,I-4,0.0,1.00,PATTY,1465.0,g
139,I-19,0.0,1.00,PATTY,135.0,g
140,I-111,0.0,2.00,PATTY,270.0,g


In [52]:
conv.loc[conv["ConversionId"] == "I-24"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
15,I-24,1.0,50.0,g,50.0,g


In [53]:
ad = pd.read_csv("data/AMS/preprocessed/Category_Assigned.csv")
ad

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom,Category
0,I-0,1.0,1000.0,g,1000.00,g,38
1,I-2,1.0,1.0,g,1.00,g,40
2,I-6,1.0,30.0,g,30.00,g,40
3,I-7,1.0,45.0,g,45.00,g,58
4,I-8,1.0,15.0,g,15.00,g,40
...,...,...,...,...,...,...,...
171,I-141,TBA,2.0,ea,2267.96,g,35
172,I-1,1.0,1000.0,g,1000.00,g,55
173,I-149,453.592,10.0,LBS,4535.92,g,4
174,I-3,0.0,2.0,PORT,4735.92,g,4


In [54]:
ad.loc[ad["ConversionId"] == "I-23"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom,Category
111,I-23,56.0,3.0,BUNC,168.0,g,40


In [22]:
outcome.loc[outcome["ConversionId"] == "I-10"]

,ConversionId,ConvertToQty,ConvertToUom,Category,CO2 Emission (g CO2),Nitrogen Lost (g),Water Withdrawals (mL),PrepId,Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),Weight (g)
57,I-10,1000.0,g,54,9.3703,0.00675,24.9,P-2,9370.3,6.75,24900.0,18579.312
58,I-10,1000.0,g,41,0.3970,0.00500,2.6,P-2,397.0,5.00,2600.0,18579.312


In [20]:
outcome[outcome.duplicated(subset=["ConversionId"], keep='first')]

,ConversionId,ConvertToQty,ConvertToUom,Category,CO2 Emission (g CO2),Nitrogen Lost (g),Water Withdrawals (mL),PrepId,Total CO2 Emission (g),Total Nitrogen Emission (g),Total Freshwater Withdrawals (mL),Weight (g)
21,I-59,39.68935,g,40,0.5029,0.0079,81.3,P-10,19.959774,0.313546,3226.744155,14176.56935
58,I-10,1000.00000,g,41,0.3970,0.0050,2.6,P-2,397.000000,5.000000,2600.000000,18579.31200
60,I-18,100.00000,g,43,1.6414,0.0000,10.1,P-3,164.140000,0.000000,1010.000000,3859.00000


In [11]:
existing_units = ["lb", "LBS", "oz", "Kg", "kg", "L"]

def convert_units(items, empty_df):
    
    def converter_constructor(value, old_unit):
        # Unit conversion
        if (old_unit == "lb") or (old_unit == "LBS"):
            value = value * 453.592
        elif old_unit == "oz":
            value = value * 28.3495
        elif (old_unit == "Kg") or (old_unit == "kg"):
            value = value * 1000
        elif old_unit == "L":
            value = value * 1000
        return (value, "g")
    
    def construct_empty_conversion_df():
        columns = ["ConversionId", "Multiplier", "ConvertFromQty", "ConvertFromUom", 
                   "ConvertToQty", "ConvertToUom"]
        df = pd.DataFrame(columns=columns)
        return df
    
    for ind, row in items.iterrows():
        conversionId = row["ItemId"]
        if row["UOM"] in ["g", "ml"]:
            fromQty = row["Qty"]
            fromUom = row["UOM"]
            toQty = row["Qty"]
            toUom = row["UOM"]
            multiplier = 1.0
        elif row["UOM"] in existing_units:
            fromQty = row["Qty"]
            fromUom = row["UOM"]
            toQty = converter_constructor(fromQty, fromUom)[0]
            toUom = converter_constructor(fromQty, fromUom)[1]
            multiplier = toQty / fromQty
        else:
            fromQty = row["Qty"]
            fromUom = row["UOM"]
            toQty = 999
            toUom = "g"
            multiplier = "TBA"
        info = [conversionId, multiplier, fromQty, fromUom, toQty, toUom]
        empty_df.loc[len(empty_df.index)] = info
        
    return empty_df